<a href="https://colab.research.google.com/github/havaledar/S_Titanic_Kaggle_6100/blob/main/6100a2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

# Ethics


# Start of Project


In [ ]:
#from google.colab import drive #files
#uploaded = files.upload()
#drive.mount('/content/drive')

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# read data
#train_data = pd.read_csv('drive/My Drive/6100/train.csv')
#test_data = pd.read_csv('drive/My Drive/6100/test.csv')
train_data = pd.read_csv('https://raw.githubusercontent.com/havaledar/S_Titanic_Kaggle_6100/main/train.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/havaledar/S_Titanic_Kaggle_6100/main/test.csv')

train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Data Cleaning

What is done so far; let me know if you want to change anything/have any questions

*   Converted 'PassengerId' into TravelGroup (1/0)
*   Converted 'Name' into TravelWithFamily (1/0)
*   Separated 'Cabin' into 3 different variables (deck/num/side), Cabin_Num is converted into numbers while the other two are kept as categories
*   One-hot encode dummy variables (drop_first=True): Homeplanet, Cable_Deck, Cable_Side, Destination
*   Change to 1/0s: CryoSleep, VIP
*   NAs: Check code block that starts with 'deal with the rest of the NA values' to see what I did with each column
*   Used train_test_split to split the data to do cross-validation


TODO:
*   M: train both models
*   Meet at night tomorrow. Discuss what to write for the report later








In [ ]:
#### Modify train and test data

# Transported is the value we want to predict
Transported = train_data['Transported']

# for data transformation

# Combine train and test
temp = [train_data, test_data]
# Remove Transported from train data to combine with test data
full_data = pd.concat(temp, keys=['train', 'test']).drop('Transported', axis=1)

#full_data.head()

In [ ]:
### convert PassengerID into travel with group (1/0)

# get their groups
full_data['Group'] = full_data['PassengerId'].str.split('_').str[0]

# count if any other people belong in that group
full_data['GroupCount'] = full_data.groupby(['Group'])['PassengerId'].transform('count')

# create TravelGroup variable
full_data['TravelGroup'] = np.where(full_data['GroupCount'] > 1, 1, 0)

#full_data.head()

In [ ]:
### convert Name into TravelWithFamily

# get their last names
full_data['LastName'] = full_data['Name'].str.split(' ').str[-1]

# create Group_LN variable (to check same group & same LN)
full_data['Group_LN'] = full_data['Group'] + full_data['LastName']

# create TravelWithFamily variable
full_data['FamilyCount'] = full_data.groupby(['Group_LN'])['PassengerId'].transform('count')
full_data['TravelWithFamily'] = np.where(full_data['FamilyCount'] > 1, 1, 0)

full_data[['Group_LN','TravelGroup','TravelWithFamily']][21:28]

Group_LN  TravelGroup  TravelWithFamily
train 21   0020Brantuarez            1                 1
      22   0020Brantuarez            1                 1
      23  0020Mcfaddennon            1                 1
      24  0020Jacostanley            1                 0
      25   0020Brantuarez            1                 1
      26  0020Mcfaddennon            1                 1
      27              NaN            0                 0

In [ ]:
### convert Cabin into deck/num/side
Cabin = full_data['Cabin'].str.split('/')

full_data['Cabin_Deck'] = Cabin.str[0]

full_data['Cabin_Num'] = Cabin.str[1]
full_data['Cabin_Num'] = pd.to_numeric(full_data['Cabin_Num'], errors='coerce')

full_data['Cabin_Side'] = Cabin.str[2]

full_data[['Cabin_Deck','Cabin_Num','Cabin_Side']].head()

Cabin_Deck  Cabin_Num Cabin_Side
train 0          B        0.0          P
      1          F        0.0          S
      2          A        0.0          S
      3          A        0.0          S
      4          F        1.0          S

In [ ]:
# convert T/F to 1/0 (fills na to False, which is 0)
tf_var = ['CryoSleep', 'VIP', 'TravelGroup', 'TravelWithFamily']
for var in tf_var:
  full_data[var] = np.where(full_data[var], 1, 0)

full_data[tf_var].head()

CryoSleep  VIP  TravelGroup  TravelWithFamily
train 0          0    0            0                 0
      1          0    0            0                 0
      2          0    1            1                 1
      3          0    0            1                 1
      4          0    0            0                 0

In [ ]:
# deal with the rest of the NA values

# take means:
na_mean_var = ['Cabin_Num', 'Age']
for var in na_mean_var:
  full_data[var].fillna(full_data[var].mean(), inplace=True)
#full_data[na_mean_var].fillna(full_data[na_mean_var].mean(), inplace=True)

# take 0s:
na_zero_var = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for var in na_zero_var:
  full_data[var].fillna(0, inplace=True)
# full_data[na_zero_var].fillna(0, inplace=True)

# make NA a category
na_cate_var = ['HomePlanet', 'Destination', 'Cabin_Deck', 'Cabin_Side']
for var in na_cate_var:
  full_data[var].fillna('NA', inplace=True)

In [ ]:
# delete columns that are unneeded after the transformation
full_data.drop(columns=['Group', 'GroupCount', 'LastName', 
                        'Name', 'Group_LN', 'FamilyCount','Cabin'], inplace=True)

In [ ]:
#one hot encode
categorical_var = ['HomePlanet', 'Destination', 'Cabin_Deck', 'Cabin_Side']

full_data = pd.get_dummies(full_data, columns=categorical_var, drop_first=True)

full_data.head()

PassengerId  CryoSleep   Age  VIP  RoomService  FoodCourt  \
train 0     0001_01          0  39.0    0          0.0        0.0   
      1     0002_01          0  24.0    0        109.0        9.0   
      2     0003_01          0  58.0    1         43.0     3576.0   
      3     0003_02          0  33.0    0          0.0     1283.0   
      4     0004_01          0  16.0    0        303.0       70.0   

         ShoppingMall     Spa  VRDeck  TravelGroup  ...  Cabin_Deck_B  \
train 0           0.0     0.0     0.0            0  ...             1   
      1          25.0   549.0    44.0            0  ...             0   
      2           0.0  6715.0    49.0            1  ...             0   
      3         371.0  3329.0   193.0            1  ...             0   
      4         151.0   565.0     2.0            0  ...             0   

         Cabin_Deck_C  Cabin_Deck_D  Cabin_Deck_E  Cabin_Deck_F  Cabin_Deck_G  \
train 0             0             0             0             0             0   
      1             0             0             0             1             0   
      2             0             0             0             0             0   
      3             0             0             0             0             0   
      4             0             0             0             1             0   

         Cabin_Deck_NA  Cabin_Deck_T  Cabin_Side_P  Cabin_Side_S  
train 0              0             0             1             0  
      1              0             0             0             1  
      2              0             0             0             1  
      3              0             0             0             1  
      4              0             0             0             1  

[5 rows x 28 columns]

In [ ]:
# split back to train and test
train_data = full_data.loc['train']
test_data = full_data.loc['test'] # this is our final test set for our results

# drop ids in training data as it's not needed
train_data = train_data.drop(columns=['PassengerId'])

# for training
x_train, x_test, y_train, y_test = train_test_split(train_data, Transported, test_size = 0.1, random_state = 20)

